# TP3 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

## Enunciado do Problema

Pretende-se construir uma implementação simplificada do algoritmo “model checking” orientado aos interpolantes seguindo a estrutura apresentada nos apontamentos onde no passo $(n,m)$ na impossibilidade de encontrar um interpolante invariante se dá ao utilizador a possibilidade de incrementar um dos índices $n$ e $m$ à sua escolha.
    
Pretende-se aplicar este algoritmo ao problema da da multiplicação de inteiros positivos em `BitVec` (apresentado no TP2).

![Algoritmo de Multiplicação](https://paper-attachments.dropboxusercontent.com/s_9896551CC5FAD2B2EB6E4EBC08522545FA66314D29FE6A5BE8E593259F8E8A37_1668181619605_multiplicacao-overflow.png)

## Análise

A Análise deste problema consiste em definir as diferenças entre o problema resolvido no TP2 e este. Principalmente, a utilização de SFOTS para o descrever, tendo em conta as diferenças nas condições de erro e na notação, tal como visto nas aulas práticas.

### Definição do SFOTS

Dessa forma, o sistema dinâmico será um tuplo:
$$ \sum \; \equiv \; <\mathcal{T},X,next,I,T,E> $$

Onde se verifica o seguinte, para representar o sistema em específico:
 1. $\mathcal{T}$ representa um SMT apropriada, que pertence à FOL, que vamos representar no nosso Solver;
 2. $X$ é o conjunto das variáveis base do problema;
 3. $next$ é um operador que gera "clones" das variáves em $X$;
 4. $I$ é um predicado unário que determina quais os estados iniciais;
 5. $T$ é um predicado binário que determina as transições entre dois estados;
 6. $E$ é um predicado unário que determina os estados de erro.

Nesse sentido, o modelo terá 3 variáveis do tipo BitVector, $x$, $y$ e $z$, este último que terminará com o resultado da execução. Para além destes, também terá uma variável inteira $p$ que representa o estado de execução. Definiu-se um inteiro para cada um dos estados, nomeadamente:
 - $0$ para Estado central (loop). Nele testam-se os casos.
 - $1$ para Estado de execução se $y$ for par (não zero).
 - $2$ para Estado de execução se $y$ for ímpar (não zero).
 - $3$ para Estado Final.
 
As cópias destas variáveis serão dadas pelo operador $next$, cuja notação se pode expandir para incluir qualquer predicado $\mathsf{P}$ que tenha $X$ como o conjunto das variáveis livres. Assim, $\,next(X) \,\equiv\, X'\,$ e $\,next(\mathsf{P}) \equiv \mathsf{P'}\equiv\mathsf{P}\,\{X\,/\,next(X)\}$ Esta notação segue a convenção das aulas teóricas, o que lhe permite ficar "livre de variáveis".

Considerando o estado inicial, manter-se-á a optimização possível à execução do autómato, isto é, definir o valor $b$ o menor dos dois. Isto diminui o número de operações totais necessárias, no pior caso. O estado inicial será então defindo pelo predicado seguinte:

$$ I \quad \equiv \quad p = 0 \wedge x = a \wedge y = b \wedge z = 0$$

O predicado de transição não terá as transições para os estados de erro, sendo indêntica à anterior nos restantes aspetos.

\begin{array}{c}
T \equiv\\
(\mathit{p} = 0 \wedge y \equiv 0 \pmod 2 \wedge y \neq 0 \wedge \mathit{p}' = 1 \wedge x' = x \wedge y' = y \wedge z' = z) \\
\vee\\
(\mathit{p} = 0 \wedge y \equiv 1 \pmod 2 \wedge y \neq 0 \wedge \mathit{p}' = 2 \wedge x' = x \wedge y' = y \wedge z' = z) \\
\vee\\
(\mathit{p} = 1 \wedge 2x < 2^n \wedge \mathit{p}' = 0 \wedge x' = 2x \wedge y' = y/2 \wedge z' = z)\\
\vee\\
(\mathit{p} = 2 \wedge 2^n - 1 - z \geq x \wedge \mathit{p}' = 0 \wedge x' = x \wedge y' = y - 1 \wedge z' = z + x)\\
\vee\\
(\mathit{p} = 0 \wedge y = 0 \wedge \mathit{p}' = 3 \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
(\mathit{p} = 3 \wedge \mathit{p}' = 3 \wedge x' = x \wedge y' = y \wedge z' = z)\\
\end{array}

Como condição de erro, considera-se as retiradas do predicado em cima descrito, mas sem a consideração do estado atual. No entanto, deve-se limitar a esta condição o valor de $y$ ser diferente de $0$, já que não se considera erro occorrer algum possível overflow se este não for efetivamenteo calculado.

\begin{array}{c}
E \equiv \\
(y \neq 0 \wedge 2x \leq 2^n)\\
\vee\\
(y \neq 0 \wedge 2^n - 1 - z < x)\\
\end{array}

Um traço de execução é uma sequência de estados, onde dois estados consequentes validam um predicado de transição. Tal como no TP2, o número de estados é finito, já que os valores que $a$ e $b$ podem tomar estão limitados pela precisão $n$, e porque as operações executadas tendem para um dos estados de *loop*, qualquer traço de execução deste problema é limitado. Dessa forma, pode-se sempre calcular o traço até ao momento em que um estado transiciona para outro que já ocorreu no traço, descrevendo assim um loop.

### Segurança e Acessibilidade

Num SFOTS $\Sigma \; \equiv \; <\mathcal{T},X,next,I,T,E>$  a  verificação deriva das noções de acessibilidade e insegurança:
- Um estado $\,r\,$ é acessível em $\,\Sigma\,$ quando $\,r\in\mathsf{I}\,$ ou quando existe uma transição $\;(s,r)\in \mathsf{T}\;$ em que $\,s\,$ é acessível em $\,\Sigma\,$.
- Um estado $\,u\,$ é inseguro em $\,\Sigma\,$ quando $\,u\in \mathsf{E}\,$ ou quando existe uma transição $\,(u,\upsilon)\in \mathsf{T}\,$ em que $\,\upsilon\,$ é inseguro em $\,\Sigma\,$.
- O SFOTS $\,\Sigma\,$ é inseguro se existe algum estado $\,s\,$ que seja simultaneamente acessível e inseguro. Em caso contrário o sistema $\,\Sigma\,$ é seguro.

Para ajudar na definição dos estados inseguros, define-se um SFOTS $\Sigma^T \; \equiv \; <\mathcal{T},Y,next,E,B,I>$, onde $Y$ é um clone das variáveis de $X$ e $B \equiv T^{-1}$.

Desses dois sistemas, definem-se os predicados $\;\mathsf{R}_n\;\equiv\; \mathsf{I}\,\land\,\mathsf{T}^n\,$ e $\mathsf{U}_m\;\equiv\; \mathsf{E} \wedge \mathsf{B}^m $ que representam traços finitos com $n$, respetivamente $m$, transições cujos estados são acessíveis, respetivamente inseguros.

Para avaliar a segurança eventual do sistema $\;\Sigma\;$ é necessário determinar se nenhum estado é simultâneamente acessível e inseguro. Para isso tem que se avaliar se, para todo o $\,n,\,m$, a fórmula  $\:\:V_{n,m}\,\equiv\,\mathsf{R}_n\,\land\,(X_n=Y_m) \,\land\,\mathsf{U}_m\:\:$ é insatisfazível, onde $\;X_n = \mathsf{top}(\mathsf{R}_n)\:\text{e}\: Y_m = \mathsf{top}(\mathsf{U}_m)$.

### Algoritmo de Interpolantes e Invariantes

## Implementação

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT, BVType

import itertools

In [2]:
n = 16

In [3]:
def bv_sel(z,i):                    # seleciona o bit i do BitVec "z"
    return BVExtract(z,start=i,end=i)

In [4]:
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),BVType(2*n))
    state['p'] = Symbol('p!'+s+str(i),INT)
    return state

In [5]:
vars = ['x','y','z']

def init1(state):
    return And(Equals(state['z'],BVZero(2*n)), Equals(state['p'], Int(0)), BVULT(state['x'], BV(2**n, 2*n)), 
               BVULT(state['y'], BV(2**n, 2*n)), BVULT(state['y'], state['x']))
    
def error1(state):
    err_odd = And(Not(Equals(state['y'], BVZero(2*n))), Equals(state['p'], Int(2)), state['x'] > BVSub(BV(2**n-1,2*n), state['z']))
    err_even = And(Not(Equals(state['y'], BVZero(2*n))), Equals(state['p'], Int(1)), Equals(bv_sel(state['x'], n-1), BVOne(1)))
    return Or(err_odd, err_even)
    
def trans1(curr, prox):
    tend = And(Equals(curr['p'], Int(0)), Equals(prox['p'], Int(3)), Equals(curr['y'], BVZero(2*n)),
               Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
    tendl = And(Equals(curr['p'], Int(3)), Equals(prox['p'], Int(3)), 
                  Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
    todd = And(Equals(curr['p'], Int(0)), Equals(prox['p'], Int(2)), Equals(bv_sel(curr['y'],0), BVOne(1)),
               Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
    toddt = And(Equals(curr['p'], Int(2)), Equals(prox['p'], Int(0)), Equals(prox['y'], curr['y'] - BVZExt(BVOne(1), 2*n-1)), 
                Equals(prox['z'], curr['z'] + curr['x']), Equals(prox['x'], curr['x']), Not(curr['x'] > BVSub(BV(2**n-1,2*n), curr['z'])))
    teven = And(Equals(curr['p'], Int(0)), Equals(prox['p'], Int(1)), Not(Equals(curr['y'], BVZero(2*n))), Equals(bv_sel(curr['y'],0), BVZero(1)),
                Equals(curr['x'], prox['x']), Equals(curr['y'], prox['y']), Equals(curr['z'], prox['z']))
    tevent = And(Equals(curr['p'], Int(1)), Equals(prox['p'], Int(0)), Equals(prox['x'], BVLShl(curr['x'], BVZExt(BVOne(1), 2*n-1))),
                 Equals(prox['y'], BVLShr(curr['y'], BVZExt(BVOne(1), 2*n-1))), Equals(curr['z'], prox['z']), Not(Equals(bv_sel(curr['x'], n-1), BVOne(1))))
    return Or(tend, tendl, todd, toddt, teven, tevent)

In [6]:
def genTrace(vars,init,trans,error,n):
    with Solver(name="z3") as s:
        
        X = [genState(vars,'X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))

In [7]:
genTrace(vars, init1, trans1, error1, 10)

Estado: 0
           x = 2056_32
           y = 2052_32
           z = 0_32
           p = 0
Estado: 1
           x = 2056_32
           y = 2052_32
           z = 0_32
           p = 1
Estado: 2
           x = 4112_32
           y = 1026_32
           z = 0_32
           p = 0
Estado: 3
           x = 4112_32
           y = 1026_32
           z = 0_32
           p = 1
Estado: 4
           x = 8224_32
           y = 513_32
           z = 0_32
           p = 0
Estado: 5
           x = 8224_32
           y = 513_32
           z = 0_32
           p = 2
Estado: 6
           x = 8224_32
           y = 512_32
           z = 8224_32
           p = 0
Estado: 7
           x = 8224_32
           y = 512_32
           z = 8224_32
           p = 1
Estado: 8
           x = 16448_32
           y = 256_32
           z = 8224_32
           p = 0
Estado: 9
           x = 16448_32
           y = 256_32
           z = 8224_32
           p = 1


In [8]:
def invert(trans):
    return (lambda curr,prox: trans(prox, curr))

In [9]:
def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

In [11]:
def model_checking(vars,init,trans,error,N,M):
    with Solver(name="msat") as s:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        #falta testar para n = 0 e m = 0
        
        for (n,m) in order:
            print((n,m))
            I = init(X[0])
            E = error(Y[0])
            Tn = And([trans(X[i], X[i+1]) for i in range(n)])
            Bm = And([invert(trans)(Y[j], Y[j+1]) for j in range(m)])
            Rn = And(I, Tn)
            Um = And(E, Bm)
            Vnm = And(Rn, Um, same(X[n], Y[m]))
            
            #1º Passo
            if s.solve([Vnm]):
                print("Unsafe!")
                for i in range(n):
                    print("Estado:",i)
                    for v in X[i]:
                        print("          ",v,'=',s.get_value(X[i][v]))
                        print("          ",v+'\'','=',s.get_value(Y[i][v]))
                return
            
            #2º Passo
            C = binary_interpolant(And(Rn, same(X[n], Y[m])), Um)
            if C is None:
                print("Interpolante None!")
                continue
            #3ª Passo
            C0 = rename(C, X[0]) #O que é que isto faz?
            C1 = rename(C, X[1])
            T = trans(X[0], X[1])
            if not s.solve([C0, T, Not(C1)]):
                print("Safe!")
                return
            
            #4º Passo - gerar o S
            S = None
            while True:
                S = rename(C, X[n])
                A = And(S, trans(X[n], Y[m]))
                if not s.solve([A, Um]):
                    S = None
                    print("Interpolante None!")
                    break
                Cnew = binary_interpolant(A, Um)
                if Cnew is None:
                    S = None
                    print("Interpolante None!")
                    break
                Cn = rename(Cnew, X[n])
                if s.solve(Implies(Cn, S)):
                    break
                S = Or(S, Cn)
            if S:
                print("Safe!")
                return
            
        print("unknown")                   

model_checking(vars, init1, trans1, error1, 50, 50)        

(1, 1)


NoSolverAvailableError: No Interpolator is available for logic QF_AUFBVLIRA